In [1]:
import pandas as pd

In [2]:
# Import game data
from game_data import game_data

In [3]:
def process_all_csv(files):
    # Concatenate all CSV files into one DataFrame
    combined_df = pd.concat([pd.read_csv(file) for file in files])

    # Filter to include only pitchers from 'UNO_MAV'
    uno_mav_df = combined_df[combined_df['PitcherTeam'] == 'UNO_MAV']

    # Group by 'Pitcher' and 'TaggedPitchType' and aggregate 'PitchCall'
    grouped_df = uno_mav_df.groupby(['Pitcher', 'TaggedPitchType'])['PitchCall'].apply(list).reset_index(name='PitchCall')

    # Calculate total pitches
    total_pitches = uno_mav_df.groupby(['Pitcher', 'TaggedPitchType']).size().reset_index(name='Times Thrown')

    # Merge the total pitches data with the grouped DataFrame
    final_df = pd.merge(grouped_df, total_pitches, on=['Pitcher', 'TaggedPitchType'], how='left')

    # Define a function to calculate the strike percentage
    def calculate_strike_percentage(pitch_calls):
        counts = pd.Series(pitch_calls).value_counts()
        total_pitches = len(pitch_calls)
        strike_types = ['InPlay', 'FoulBall', 'FoulBallNotFieldable', 'StrikeCalled', 'StrikeSwinging']
        strike_count = sum(counts.get(pitch_type, 0) for pitch_type in strike_types)
        return (strike_count / total_pitches) * 100 if total_pitches else 0

    # Apply the function to get the 'Strike%' column
    final_df['Strike%'] = final_df['PitchCall'].apply(calculate_strike_percentage)

    # Calculate 'Whiff Rate'
    final_df['Whiff Rate'] = final_df['PitchCall'].apply(lambda x: 
                             (x.count('StrikeSwinging') / 
                             (x.count('InPlay') + x.count('FoulBall') + x.count('StrikeSwinging')) * 100 
                              if (x.count('InPlay') + x.count('FoulBall') + x.count('StrikeSwinging')) > 0 else 0))

    # Handle NaN and round percentages
    final_df['Strike%'] = final_df['Strike%'].fillna(0).round(0).astype(int).astype(str) + '%'
    final_df['Whiff Rate'] = final_df['Whiff Rate'].fillna(0).round(0).astype(int).astype(str) + '%'

    # Remove rows where Times Thrown is less than 10
    final_df = final_df[final_df['Times Thrown'] >= 10]
    
    # Rename the columns as needed
    final_df = final_df.rename(columns={
        'TaggedPitchType': 'Pitch Type'
    })

    # Select only the required columns for the final output
    final_df = final_df[['Pitcher', 'Pitch Type', 'Times Thrown', 'Strike%', 'Whiff Rate']]

    # Export the DataFrame to a CSV file
    final_df.to_csv('Pitch_Control_Analysis.csv', index=False)

    return final_df
   

In [4]:
# Process all CSV files and get the final percentage DataFrame
final_df = process_all_csv(game_data)

final_df.head(50)

/var/folders/2c/tr3ggy4904g5pqdkqt7wqyym0000gn/T/ipykernel_67076/603331778.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([pd.read_csv(file) for file in files])


,Pitcher,Pitch Type,Times Thrown,Strike%,Whiff Rate
0,"Bell, Charlie",ChangeUp,112,68%,54%
1,"Bell, Charlie",Fastball,221,61%,32%
2,"Bell, Charlie",FourSeamFastBall,40,57%,56%
4,"Bell, Charlie",Slider,76,71%,26%
5,"Byhre, Chris",ChangeUp,18,33%,75%
6,"Byhre, Chris",Fastball,37,57%,29%
7,"Byhre, Chris",FourSeamFastBall,14,43%,50%
11,"Curtis, Brayden",Fastball,42,60%,36%
13,"Curtis, Brayden",Slider,60,58%,50%
14,"Dreher, Matt",ChangeUp,11,82%,60%
